In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import sys, copy, os, shutil
from tqdm.notebook import tqdm

# load in our true case counts
cases = pd.read_csv("processed/weekly_cases.csv")
cases.date = pd.to_datetime(cases.date)
cases.set_index("date", inplace=True)

# our set of locations
locations = pd.read_csv("processed/locations.csv")

# color-blind friendly colors
colors = ['#377eb8', '#ff7f00', '#4daf4a',
          '#f781bf', '#a65628', '#984ea3',
          '#999999', '#e41a1c', '#dede00']

In [2]:
# what columns are we interested in?
para_cols = ["model", "fh", "log_trans", "num_nodes", "cluster_mech", 
             "num_lags", "shuffle", "local_reg", "neighbor_reg"]
mMAE_cols = [f"mMAE_{loc}" for loc in locations.location_key.values] + ["mMAE_tot"]
sMAE_cols = [f"sMAE_{loc}" for loc in locations.location_key.values] + ["sMAE_tot"]

# create our dataframe 
logs = pd.DataFrame(data=None, columns=para_cols + mMAE_cols + sMAE_cols)

# let's use a common test set for all horizons just to keep things fair and comparable
pred_start_shared = pd.Timestamp(2021, 2, 21)

In [3]:
# iterate thru all TGCN trials
for fname in tqdm(sorted([f for f in os.listdir("results/tgcn") if ".csv" in f])):
    
    # specify what model we're using
    model = "tgcn"
    
    # unpack the settings for TGCN
    fh, log_trans, num_nodes, cluster_mech, num_lags, shuffle = tuple([s.split("=")[1] \
                                                                       for s in fname\
                                                                       .replace(".csv", "").split("_")])

    # convert to appropriate data types
    fh, num_nodes, cluster_mech, num_lags = int(fh), int(num_nodes), int(cluster_mech), int(num_lags)
    log_trans = True if log_trans == "True" else False
    shuffle = True if shuffle == "True" else False
    
    # load our predictions, too + extract out the start and end
    preds = pd.read_csv(f"results/tgcn/{fname}")
    preds.date = pd.to_datetime(preds.date)
    preds.set_index("date", inplace=True)
    pred_start, pred_end = preds.index[0], preds.index[-1]
    
    # overwrite the pred_start + truncate
    pred_start = pred_start_shared
    preds = preds.loc[pred_start_shared :]
    
    # compute the naive predictions
    naive_start = pred_start - relativedelta(days=7*fh)
    naive_end = pred_end - relativedelta(days=7*fh)
    naive_preds = cases.loc[naive_start : naive_end].set_index(preds.index)
    
    # compute scaled MAE (justify because don't want to penalize overprediction egregiously, as in MSE)
    truth = cases.loc[pred_start : pred_end]
    nMAEs = np.abs(truth - naive_preds).mean() # naive MAE
    mMAEs = np.abs(truth - preds).mean() # model MAE
    sMAEs = mMAEs / nMAEs # the scaled version -- how much better do we do than persistence?
    
    # for cross-compatibility with the linear models
    local_reg, neighbor_reg = np.nan, np.nan
    
    # store both the scaled and unscaled versions!
    row = [model, fh, log_trans, num_nodes, cluster_mech, num_lags, shuffle, local_reg, neighbor_reg]
    row += (list(mMAEs.values) + [mMAEs.mean()])
    row += (list(sMAEs.values) + [sMAEs.mean()])
    logs.loc[len(logs.index)] = row

  0%|          | 0/5040 [00:00<?, ?it/s]

In [4]:
# iterate thru all DCRNN trials
for fname in tqdm(sorted([f for f in os.listdir("results/dcrnn") if ".csv" in f])):
    
    # specify what model we're using
    model = "dcrnn"
    
    # unpack the settings for DCRNN
    fh, log_trans, num_nodes, cluster_mech, num_lags, shuffle = tuple([s.split("=")[1] \
                                                                       for s in fname\
                                                                       .replace(".csv", "").split("_")])

    # convert to appropriate data types
    fh, num_nodes, cluster_mech, num_lags = int(fh), int(num_nodes), int(cluster_mech), int(num_lags)
    log_trans = True if log_trans == "True" else False
    shuffle = True if shuffle == "True" else False
    
    # load our predictions, too + extract out the start and end
    preds = pd.read_csv(f"results/dcrnn/{fname}")
    preds.date = pd.to_datetime(preds.date)
    preds.set_index("date", inplace=True)
    pred_start, pred_end = preds.index[0], preds.index[-1]
    
    # overwrite the pred_start + truncate
    pred_start = pred_start_shared
    preds = preds.loc[pred_start_shared :]
    
    # compute the naive predictions
    naive_start = pred_start - relativedelta(days=7*fh)
    naive_end = pred_end - relativedelta(days=7*fh)
    naive_preds = cases.loc[naive_start : naive_end].set_index(preds.index)
    
    # compute scaled MAE (justify because don't want to penalize overprediction egregiously, as in MSE)
    truth = cases.loc[pred_start : pred_end]
    nMAEs = np.abs(truth - naive_preds).mean() # naive MAE
    mMAEs = np.abs(truth - preds).mean() # model MAE
    sMAEs = mMAEs / nMAEs # the scaled version -- how much better do we do than persistence?
    
    # for cross-compatibility with the linear models
    local_reg, neighbor_reg = np.nan, np.nan
    
    # store both the scaled and unscaled versions!
    row = [model, fh, log_trans, num_nodes, cluster_mech, num_lags, shuffle, local_reg, neighbor_reg]
    row += (list(mMAEs.values) + [mMAEs.mean()])
    row += (list(sMAEs.values) + [sMAEs.mean()])
    logs.loc[len(logs.index)] = row

  0%|          | 0/5040 [00:00<?, ?it/s]

In [5]:
# iterate thru all linear trials
for fname in tqdm(sorted([f for f in os.listdir("results/linear") if ".csv" in f])):
    
    # specify what model we're using
    model = "linear"
    
    # unpack the settings for our linear one
    fh, log_trans, num_lags, cluster_mech, reg_scheme = tuple([s.split("=")[1] \
                                                               for s in fname\
                                                               .replace(".csv", "").split("_")])

    # convert to appropriate types
    fh, num_lags, cluster_mech = int(fh), int(num_lags), int(cluster_mech)
    log_trans = True if log_trans == "True" else False

    # unpack the local + neighbor regularization factors
    local_reg, neighbor_reg = np.array(reg_scheme.split("+")).astype(int)
    
    # load our predictions, too + extract out the start and end
    preds = pd.read_csv(f"results/linear/{fname}")
    preds.date = pd.to_datetime(preds.date)
    preds.set_index("date", inplace=True)
    pred_start, pred_end = preds.index[0], preds.index[-1]
    
    # overwrite the pred_start + truncate
    pred_start = pred_start_shared
    preds = preds.loc[pred_start_shared :]
    
    # compute the naive predictions
    naive_start = pred_start - relativedelta(days=7*fh)
    naive_end = pred_end - relativedelta(days=7*fh)
    naive_preds = cases.loc[naive_start : naive_end].set_index(preds.index)
    
    # compute scaled MAE (justify because don't want to penalize overprediction egregiously, as in MSE)
    truth = cases.loc[pred_start : pred_end]
    nMAEs = np.abs(truth - naive_preds).mean() # naive MAE
    mMAEs = np.abs(truth - preds).mean() # model MAE
    sMAEs = mMAEs / nMAEs # the scaled version -- how much better do we do than persistence?
    
    # for cross-compatibility with the graph neural network models
    num_nodes, shuffle = np.nan, np.nan
    
    # store both the scaled and unscaled versions!
    row = [model, fh, log_trans, num_nodes, cluster_mech, num_lags, shuffle, local_reg, neighbor_reg]
    row += (list(mMAEs.values) + [mMAEs.mean()])
    row += (list(sMAEs.values) + [sMAEs.mean()])
    logs.loc[len(logs.index)] = row

  0%|          | 0/4032 [00:00<?, ?it/s]

In [6]:
# save our logs files
logs.to_csv("performance_logs.csv", index=False)